In [1]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

In [2]:
data_route = "C:\\workspace\\project_final\\data\\dataset_sample_10percent.csv"
data = pd.read_csv(data_route, encoding='utf-8')

In [3]:
# Z-score 계산 및 이상치 여부 확인 함수
def calculate_z_scores(df, group_column):
    df[f'{group_column}_zscore'] = df.groupby(group_column)['price'].transform(lambda x: zscore(x, ddof=0))
    return df

data['brandclass_name'] = data['brand_name']+ '_' +data['class_name'] 

# 기준 컬럼별로 Z-score 계산 및 이상치 판단
data = calculate_z_scores(data, 'supplier_code')
data = calculate_z_scores(data, 'brandclass_name')
data = calculate_z_scores(data, 'cate2_nm')

In [4]:
z_threshold = 3
data['overall_outlier'] = (
    (data['supplier_code_zscore'] > z_threshold) & 
    (data['brandclass_name_zscore'] > z_threshold) & 
    (data['cate2_nm_zscore'] > z_threshold)
)

In [5]:
outliers = data[data['overall_outlier'] == True]

# 필요한 컬럼만 선택해서 출력
display(outliers[['prd_id', 'supplier_code', 'cate2_nm', 'class_name', 'brand_name', 
                  'supplier_code_zscore', 'brandclass_name_zscore', 'cate2_nm_zscore',
                  'overall_outlier', 'price']])

print(len(outliers)/len(data)*100, "% outlier based on Z-score")

,prd_id,supplier_code,cate2_nm,class_name,brand_name,supplier_code_zscore,brandclass_name_zscore,cate2_nm_zscore,overall_outlier,price
4206,1037438760,1020872,DIY/반제품가구,공간박스,오너클랜,15.359703,4.538480,5.636001,True,1435400
5344,93430532,1034195,책장/책꽂이,책장,베이직가구,3.486194,3.139458,19.859343,True,13490000
5547,1036855405,1034195,드레스룸/행거,이불장/옷장,베이직가구,8.297176,3.782171,29.769558,True,27350000
6667,50443268,1034195,식탁,식탁,베이직가구,3.548675,3.071473,11.492056,True,13670000
7331,1016357855,1043876,테이블,식탁세트,벤티스가구,4.202326,3.294012,5.331418,True,4041000
...,...,...,...,...,...,...,...,...,...,...
1588445,27808695,1031568,휴대폰케이스,핸드폰 액세서리,쇼베뉴엘,3.218066,3.211342,11.486311,True,116600
1589049,95070697,1036305,휴대폰케이스,핸드폰 액세서리,VERAX,3.193751,3.217321,3.344037,True,44300
1589891,1060780773,1040823,휴대폰케이스,핸드폰 액세서리,테일즈,3.087230,3.087230,3.073754,True,41900
1590011,1017439948,1036305,휴대폰케이스,핸드폰 액세서리,VERAX,3.967197,3.995334,4.087315,True,50900


0.15931739782421184 % outlier based on Z-score


In [6]:
# 종합 점수 계산 함수 - 5 대신 적당히 조정해도 됨
def calculate_weighted_score(df):
    df['supplier_score'] = 1 - 2*np.exp(-df['supplier_code_zscore'].abs())
    df['brandclass_score'] = 1 - 2*np.exp(-df['brandclass_name_zscore'].abs())
    df['cate2_score'] = 1 - 2*np.exp(-df['cate2_nm_zscore'].abs())
    
    # 전체 종합 점수 계산
    df['outlier_score'] = (df['supplier_score'] * df['brandclass_score'] * df['cate2_score'])*100
    return df

In [29]:
data = calculate_weighted_score(data)
outliers = data[data['overall_outlier'] == True]
outliers = outliers.sort_values(by='outlier_score', ascending=False)

In [31]:
display(outliers[['prd_id', 'supplier_code', 'cate2_nm', 'class_name', 'brand_name', 
                  'overall_outlier', 'price', 'supplier_score'	,'brandclass_score'	,'cate2_score'	,'outlier_score']])

,prd_id,supplier_code,cate2_nm,class_name,brand_name,overall_outlier,price,supplier_score,brandclass_score,cate2_score,outlier_score
376431,1035498614,1046582,국내도서,소설/문학/취미/종교/역사,웅진북센,True,1350000,1.000000,1.000000,1.000000,100.000000
503641,1049570291,1050625,필기구,연필/볼펜/기타필기구,기타브랜드,True,8803000,1.000000,1.000000,1.000000,100.000000
1244645,1063947227,1048588,냄비,주방용품 기타,기타브랜드,True,3510500,1.000000,1.000000,1.000000,100.000000
484021,1048141236,1020872,팬시용품,연필/볼펜/기타필기구,오너클랜,True,8706300,1.000000,1.000000,1.000000,100.000000
235946,1065102008,1052375,전기/철물자재,휴대용 전동공구,기타브랜드,True,586403700,1.000000,1.000000,1.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...
1169614,1025652878,1048560,커피용품,주방용품 기타,기타브랜드,True,287620,0.940188,0.948207,0.902465,80.454162
203677,60560624,1035535,접착제/테이프/본드,개인보호용품,나선산업안전,True,307000,0.917174,0.903977,0.970328,80.450320
365220,1061873961,1048384,국내도서,소설/문학/취미/종교/역사,도서기타,True,108000,0.903385,0.961963,0.924611,80.350788
43634,1066557941,1044568,여성가방,크로스백,Alexander McQueen,True,5150000,0.959957,0.910855,0.918208,80.286511
